In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [4]:
import pandas as pd
import numpy as np
import bs4
import lxml.etree as xml
import time
import datetime
from tqdm import tqdm
import random

In [5]:
PATH = '/Users/Asus/Downloads/chromedriver' # Defines where my cromedriver package is installed. Opens an automated Chrome window.
driver = webdriver.Chrome(PATH) # names webdriver as driver and assigns path. 
driver.quit() # closes the driver so the window is not always on. 

In [6]:
url = 'https://www.bolha.com/nepremicnine/lokacija-stanovanja-studenci-90.00-m2-oglas-5298888'

In [7]:
driver = webdriver.Chrome(PATH) 
driver.get(url) # automatically opens Nepremicnine.net
# driver.quit() # this closes the window. 
html = driver.page_source
webPage = bs4.BeautifulSoup(html, "lxml")
driver.quit()

###  What can be scraped from each ad?
#### Nepremicnine ads: 
From more info: (4)
- type (stanovanje, hiša..)
- region (LJ-Mesto, Gorenjska...)
- upravna enota (Lj. Bežigrad..)
- municipality (LJubljana) 
From opis: 
kratek: (7)
- size(m2)
- class (3- sobno..)
- built (leto izgradnje) 
- renovated (leto adaptirano)
- position (2/10... nadstropje..)
- price
- price-old
slika: 
- dmiss (label Ne spreglejte -yes/no)
potential: 
- number of pictures( from: .../galerija/) 


#### Bolha ads: 
- price (from li class:'price-item price-item--base'>
ul class ='meta-items'>: (3)
- posted (date)
- lasts ( objavljen se.)
- t_shown (prikazov:)
tbody (tabela s podatki): 
 - price_type 




 


In [31]:
d_df = pd.DataFrame.from_dict([d])
d_df

,Vrsta cene,Posredovanje,Lokacija,Umeščenost stanovanja,Nadstropje,Vrsta stanovanja,Bivalna površina,Neto površina,Leto izgradnje*,Balkon / terasa / atrij,Interna številka oglasa
0,Skupna cena,Prodam,"Osrednjeslovenska, Ivančna Gorica, Ivančna Gorica",V večstanovanjski stavbi,Pritličje,3-sobno,"61,50 m²","66,00 m²",2005,Balkon,1052


In [36]:

a = {}
for item in webPage.find_all('tr'):
    f_names = item.find('th')
    f_atr = item.find('td')
    a[f_names.text.replace(':', '')] = f_atr.text
    #print(f_names.text)
    #print(f_atr.text)
print(a)
print('---------')

{'Vrsta cene': 'Skupna cena', 'Posredovanje': 'Prodam', 'Lokacija': 'Podravska, Maribor, Studenci', 'Umeščenost stanovanja': 'V večstanovanjski stavbi', 'Nadstropje': '2.', 'Skupno število nadstropij v stavbi': '3', 'Vrsta stanovanja': '4,5-sobno', 'Bivalna površina': '90,30 m²', 'Neto površina': '98,40 m²', 'Leto izgradnje*': '2020', 'Opremljenost in stanje': 'Delno opremljeno', 'Balkon / terasa / atrij': 'Balkon, Loža', 'Energetski razred': 'B2 (25 - 35 kWh/m2a)'}
---------


In [133]:
# adds two different DataFrames together and inputs nan where they do not overlap in columns :D 
# This discovery makes me very happy.
data = pd.concat([d_df,a_df], axis=0, ignore_index=True)
data

,Vrsta cene,Posredovanje,Lokacija,Umeščenost stanovanja,Nadstropje,Vrsta stanovanja,Bivalna površina,Neto površina,Leto izgradnje*,Balkon / terasa / atrij,Interna številka oglasa,Skupno število nadstropij v stavbi,Opremljenost in stanje,Energetski razred
0,Skupna cena,Prodam,"Osrednjeslovenska, Ivančna Gorica, Ivančna Gorica",V večstanovanjski stavbi,Pritličje,3-sobno,"61,50 m²","66,00 m²",2005,Balkon,1052,NaN,NaN,NaN
1,Skupna cena,Prodam,"Podravska, Maribor, Studenci",V večstanovanjski stavbi,2.,"4,5-sobno","90,30 m²","98,40 m²",2020,"Balkon, Loža",NaN,3,Delno opremljeno,B2 (25 - 35 kWh/m2a)


## Scrapes individual ads' content for info.

In [8]:
def scrape_content(page):
    a = {}
    
    # adding details about the ad (first published, times viewed...)
    ad_info = page.find('ul', class_ = 'meta-items')

    for item in ad_info.find_all('li'):   
        f_names = item.find('span', class_='label').text
        f_atr = item.find(class_='value').text

        a[f_names.replace(':', '')] = f_atr.replace(' ', '')

    # adding property price
    div_price = page.find('ul', class_ = 'price-items cf')
    f_names= div_price.find(class_='label').text
    f_price_raw =  div_price.find(class_='price price--hrk').text.replace(' ', '')
    
    f_atr = f_price_raw
    #f_price_raw if str(f_price_raw) == 'podogovoru' else f_price_raw.split('\n')[2].replace(u'\xa0', u' ')

    a[f_names.replace(':', '')] = f_atr

    # adding info about the property
    table_info = page.find('tbody')

    for item in table_info.find_all('tr'):
        f_names = item.find('th')
        f_atr = item.find('td')

        a[f_names.text.replace(':', '')] = f_atr.text
        
    # make dict a into a df. 
    a_df = pd.DataFrame.from_dict([a])
        
    return a_df


In [9]:
url_source = webPage

In [10]:
scrape_ads(url_source)

NameError: name 'scrape_ads' is not defined

In [11]:
# adding df of every page to the main df.
def append_dfs(mainDF, newDF):
    df = pd.concat([mainDF,newDF], axis=0, ignore_index=True)
    return df

In [12]:
url_base = 'https://www.bolha.com/prodaja-stanovanja'

In [13]:
def getHTML_one(url):
    driver = webdriver.Chrome(PATH) 
    driver.get(url) # automatically opens Nepremicnine.net
    # driver.quit() # this closes the window. 
    html = driver.page_source
    driver.quit()
    webPage = bs4.BeautifulSoup(html, "lxml")
   
    return webPage

In [14]:
def getHTML(url):
    driver.get(url)
    html = driver.page_source
    webPage = bs4.BeautifulSoup(html, "lxml")
   
    return webPage

In [8]:
page = getHTML_one('https://www.bolha.com/prodaja-stanovanja?geo%5BlocationIds%5D=26325%2C26324%2C26323%2C26322%2C26321%2C26320%2C26319%2C26318%2C26317%2C26316%2C26315%2C26314%2C44285%2C30277&typeOfTransaction=sell&page=48')

In [15]:
def get_first_HTML(url):
    driver.get(url)
    html = driver.page_source
    webPage = bs4.BeautifulSoup(html, "lxml")
   
    return webPage

In [16]:
print([*data])

NameError: name 'data' is not defined

In [17]:
def ads_cutout(page):
    num = page.find('strong', class_="entities-count").text
    soup = page.find('div', class_='EntityList EntityList--Standard EntityList--Regular EntityList--ListItemRegularAd EntityList--itemCount_'+str(num)).prettify()
    ads = soup.find_all('h3', class_= 'entity-title')
    return ads

In [18]:
def scrape_links(page): 
    d = {'Ad_ID':[], 'Ad_Link':[], 'datetime' : []}
    num = page.find('strong', class_="entities-count").text
    soup = page.find('div', class_='EntityList EntityList--Standard EntityList--Regular EntityList--ListItemRegularAd EntityList--itemCount_'+str(num))
    ads = soup.find_all('h3', class_= 'entity-title')
    
    for ad in ads: 
        ad_link= ad.find('a', class_= 'link')['href']
        ad_id = ad_link.split('-')[-1]
        d['Ad_ID'].append(ad_id)
        d['Ad_Link'].append(ad_link)
        d['datetime'].append(datetime.datetime.now().strftime("%d.%m.%Y_%H:%M:%S"))
    #print(d)    
    d_df = pd.DataFrame(d,)
    print(d_df)
    return d_df

In [21]:
# url for all flats listed in Slovenia
url_b = 'https://www.bolha.com/prodaja-stanovanja?geo%5BlocationIds%5D=26325%2C26324%2C26323%2C26322%2C26321%2C26320%2C26319%2C26318%2C26317%2C26316%2C26315%2C26314%2C44285%2C30277&typeOfTransaction=sell&page=' 
# add page number at the end

In [22]:
def total_pages(page):
    t_ads = int(page.find('strong', class_='entities-count').text)
    t_pages = round(((t_ads / 25) +1))
    return t_pages

## Scraping all available ads and corresponding links. 

In [23]:
# working version

driver = webdriver.Chrome(PATH) 
base_url = url_b

page_base = get_first_HTML(base_url)
start = 1
end =  total_pages(page_base)
data = pd.DataFrame()


for i in tqdm(range(start,end +1)):
    urli = base_url + str(i)
    page = getHTML(urli)
    i_df = scrape_links(page)
    #print(i_df)
    data = pd.concat([data, i_df], axis=0, ignore_index=True)
              
driver.quit()

#data.to_csv("./bolha_flats_" + str(start) + '-'+ str(end) + '_' + str(date) + ".csv")

  2%|█▌                                                                                 | 1/54 [00:01<00:54,  1.02s/it]

      Ad_ID                                            Ad_Link  \
0   7006251  /nepremicnine/prodaja-stanovanje-trisobno-podr...   
1   7006250  /nepremicnine/prodaja-stanovanje-trisobno-podr...   
2   6677209  /nepremicnine/lokacija-stanovanja-zagorje-ob-s...   
3   7006122  /nepremicnine/stanovanje-v-dvojcku-vrstna-hisa...   
4   7005502  /nepremicnine/lokacija-stanovanja-limbus-70.00...   
5   3971372  /nepremicnine/lokacija-stanovanja-mala-kostrev...   
6   7001010  /nepremicnine/prodaja-stanovanje-stirisobno-ju...   
7   6191963  /nepremicnine/lokacija-stanovanja-ljutomer-125...   
8   6191828  /nepremicnine/lokacija-stanovanja-ljutomer-158...   
9   3838389  /nepremicnine/lokacija-stanovanja-center-41.00...   
10  4842815  /nepremicnine/lokacija-stanovanja-moravske-top...   
11  7000409  /nepremicnine/lokacija-stanovanja-slovenj-grad...   
12  4440382  /nepremicnine/prijetno-dvosobno-stanovanje-odl...   
13  3857126  /nepremicnine/lokacija-stanovanja-borova-vas-6...   
14  699932

  4%|███                                                                                | 2/54 [00:01<00:50,  1.03it/s]

      Ad_ID                                            Ad_Link  \
0   6837730  /nepremicnine/lokacija-stanovanja-kosaki-113.0...   
1   6668069  /nepremicnine/lokacija-stanovanja-kosaki-98.00...   
2   6667945  /nepremicnine/lokacija-stanovanja-kosaki-95.00...   
3   6667913  /nepremicnine/maribor-kosaki-novogradnja-3-sob...   
4   6617291  /nepremicnine/maribor-studenci-tabor-novogradn...   
5   6616201  /nepremicnine/lokacija-stanovanja-kosaki-128.0...   
6   6608652  /nepremicnine/maribor-kosaki-novogradnja-4-sob...   
7   3577634  /nepremicnine/maribor-studenci-106.00-m2-4-sob...   
8   3561472  /nepremicnine/maribor-studenci-104.00-m2-novog...   
9   3161746  /nepremicnine/lokacija-stanovanja-studenci-189...   
10  6716895  /nepremicnine/lokacija-stanovanja-trzaska-86-o...   
11  6995793  /nepremicnine/opatija-novogradnja-1s-db-oglas-...   
12  6995777  /nepremicnine/atraktivna-novogradnja-1-red-mor...   
13  6995775  /nepremicnine/moderno-opremljen-stan-najljepso...   
14  699577

  6%|████▌                                                                              | 3/54 [00:02<00:48,  1.06it/s]

...

      Ad_ID                                            Ad_Link  \
0   4251314     /nepremicnine/stanovanje-njivice-oglas-4251314   
1   4240433  /nepremicnine/prodamo-2-sobno-stanovanje-senti...   
2   4237513  /nepremicnine/prodaja-stanovanje-porec-vabriga...   
3   4226268  /nepremicnine/3-sobno-stanovanje-107.30-m2-z-l...   
4   4221312  /nepremicnine/3-sobno-stanovanje-111.50-m2-z-l...   
5   4182905  /nepremicnine/novigrad-2-sobno-stanovanje-z-ba...   
6   4157943  /nepremicnine/piran-lucija-stanovanje-5-in-vec...   
7   4157327   /nepremicnine/stanovanje-vantacici-oglas-4157327   
8   4157319         /nepremicnine/stanovanje-rab-oglas-4157319   
9   4157318       /nepremicnine/stanovanje-opric-oglas-4157318   
10  4157315       /nepremicnine/stanovanje-selce-oglas-4157315   
11  4157309  /nepremicnine/stanovanje-vinodolski-oglas-4157309   
12  4157307         /nepremicnine/stanovanje-krk-oglas-4157307   
13  4155040  /nepremicnine/ljubljana-moste-stanovanje-1-sob...   
14  415502

 98%|████████████████████████████████████████████████████████████████████████████████▍ | 53/54 [00:49<00:00,  1.15it/s]

      Ad_ID                                            Ad_Link  \
0   4061271  /nepremicnine/stanovanja-garsonjera-1-sobno-z-...   
1   4061270  /nepremicnine/vecje-1-sobno-stanovanje-z-galer...   
2   4061258  /nepremicnine/v-celoti-obnovljeno-3-sobno-stan...   
3   4053192  /nepremicnine/kranj-levstikova-ulica-oglas-405...   
4   4042272  /nepremicnine/ljubljana-bezigrad-dunajski-kris...   
5   4022609  /nepremicnine/stanovanje-potrebno-obnove-proda...   
6   4022408  /nepremicnine/prodaja-stanovanje-stirisobno-po...   
7   4022106  /nepremicnine/prodaja-stanovanje-stirisobno-lj...   
8   4021936  /nepremicnine/prodaja-stanovanje-dvosobno-savi...   
9   4021934  /nepremicnine/prodaja-stanovanje-dvosobno-dva-...   
10  4021933  /nepremicnine/prodaja-stanovanje-dvosobno-savi...   
11  4021486  /nepremicnine/prodaja-stanovanje-dvosobno-podr...   
12  4021324  /nepremicnine/prodaja-stanovanje-dupleks-savin...   
13  4021210     /nepremicnine/2-sobno-stanovanje-oglas-4021210   
14  402120

100%|██████████████████████████████████████████████████████████████████████████████████| 54/54 [00:50<00:00,  1.07it/s]

      Ad_ID                                            Ad_Link  \
0   2684982  /nepremicnine/lokacija-stanovanja-bled-29.80-m...   
1   3702662  /nepremicnine/lokacija-stanovanja-hoce-55.00-m...   
2   3499787  /nepremicnine/lokacija-stanovanja-lendava-57.7...   
3   2637110  /nepremicnine/lokacija-stanovanja-umag-56.00-m...   
4   2222020  /nepremicnine/stanovanje-gorenjska-cerklje-gor...   
5   2786494  /nepremicnine/lokacija-stanovanja-center-102.0...   
6   2219287  /nepremicnine/stanovanje-1.5-sobno-ruse-39.2-m...   
7   2217657  /nepremicnine/stanovanje-pomurska-murska-sobot...   
8   2221094  /nepremicnine/stanovanje-pomurska-murska-sobot...   
9   2222326  /nepremicnine/stanovanje-osrednjeslovenska-lju...   
10  2218508  /nepremicnine/stanovanje-savinjska-zrece-3-sob...   

               datetime  
0   02.05.2021_11:29:55  
1   02.05.2021_11:29:55  
2   02.05.2021_11:29:55  
3   02.05.2021_11:29:55  
4   02.05.2021_11:29:55  
5   02.05.2021_11:29:55  
6   02.05.2021_11:29:55  

In [24]:
data

,Ad_ID,Ad_Link,datetime
0,7006251,/nepremicnine/prodaja-stanovanje-trisobno-podr...,02.05.2021_11:29:06
1,7006250,/nepremicnine/prodaja-stanovanje-trisobno-podr...,02.05.2021_11:29:06
2,6677209,/nepremicnine/lokacija-stanovanja-zagorje-ob-s...,02.05.2021_11:29:06
3,7006122,/nepremicnine/stanovanje-v-dvojcku-vrstna-hisa...,02.05.2021_11:29:06
4,7005502,/nepremicnine/lokacija-stanovanja-limbus-70.00...,02.05.2021_11:29:06
...,...,...,...
1331,2219287,/nepremicnine/stanovanje-1.5-sobno-ruse-39.2-m...,02.05.2021_11:29:55
1332,2217657,/nepremicnine/stanovanje-pomurska-murska-sobot...,02.05.2021_11:29:55
1333,2221094,/nepremicnine/stanovanje-pomurska-murska-sobot...,02.05.2021_11:29:55
1334,2222326,/nepremicnine/stanovanje-osrednjeslovenska-lju...,02.05.2021_11:29:55


In [25]:
date = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
data.to_csv("./bolha_flats_" + str(start) + '-'+ str(end) + '_' + str(date) + ".csv")

In [17]:
data1  = data[:50]
data1.head(5)

,Ad_ID,Ad_Link,datetime
0,5784474,/nepremicnine/lokacija-stanovanja-center-125.0...,29.03.2021_12:40:16
1,6654426,/nepremicnine/prodamo-obnovljeno-1.5-sobno-sta...,29.03.2021_12:40:16
2,6624845,/nepremicnine/lokacija-stanovanja-kosaki-62.00...,29.03.2021_12:40:16
3,6624148,/nepremicnine/lokacija-stanovanja-kosaki-68.00...,29.03.2021_12:40:16
4,6617291,/nepremicnine/maribor-studenci-tabor-novogradn...,29.03.2021_12:40:16


## Scraping individual ads into a DataFrame

In [29]:
# just added the function def, not tested yet. Integrate per-partes scraping for up to given limit (default = 100?)

def scraping_ads(df):
    driver = webdriver.Chrome(PATH) 
    data_ads = pd.DataFrame()
    
    base_url = 'https://www.bolha.com'

    for i in tqdm(range(0,len(df))):
        urli = base_url+str(df['Ad_Link'][i])
        page = getHTML(urli)
        i_df = scrape_content(page)

        data_ads = pd.concat([data_ads, i_df], axis=0, ignore_index=True)
        sleep_time = abs(np.random.normal(0.3,0.1))
        time.sleep(sleep_time)

    driver.quit()
    return data_ads

In [ ]:
data_all = pd.read_csv('bolha_flats_1-47_29-03-2021_13-11-20.csv')
data_all= data_all.drop('Unnamed: 0', axis = 1)
data_all.head()

In [56]:
data.iloc[5]

Ad_ID                                                 3971372
Ad_Link     /nepremicnine/lokacija-stanovanja-mala-kostrev...
datetime                                  02.05.2021_11:29:06
Name: 5, dtype: object

In [81]:
# selecting only ads that have not yet been scraped previously
#def select_new(df_all, df):
data_new = data.

for i in range(len(data)): 
    ad_id = data.iloc[i]['Ad_ID']
    #print(data.iloc[i]['Ad_ID'])
    if data_all['Ad_ID'].isin([ad_id]).any() == True:
        print(data.iloc[i])
        #data_new.append(data.iloc[i])

Ad_ID                                                 3971372
Ad_Link     /nepremicnine/lokacija-stanovanja-mala-kostrev...
datetime                                  02.05.2021_11:29:06
Name: 5, dtype: object
Ad_ID                                                 6191963
Ad_Link     /nepremicnine/lokacija-stanovanja-ljutomer-125...
datetime                                  02.05.2021_11:29:06
Name: 7, dtype: object
Ad_ID                                                 6191828
Ad_Link     /nepremicnine/lokacija-stanovanja-ljutomer-158...
datetime                                  02.05.2021_11:29:06
Name: 8, dtype: object
Ad_ID                                                 3838389
Ad_Link     /nepremicnine/lokacija-stanovanja-center-41.00...
datetime                                  02.05.2021_11:29:06
Name: 9, dtype: object
Ad_ID                                                 4842815
Ad_Link     /nepremicnine/lokacija-stanovanja-moravske-top...
datetime                                

Ad_ID                                                 3211042
Ad_Link     /nepremicnine/stanovanje-savinjska-celje-vas-t...
datetime                                  02.05.2021_11:29:28
Name: 587, dtype: object
Ad_ID                                                 1470990
Ad_Link     /nepremicnine/stanovanje-pomurska-lendava-osta...
datetime                                  02.05.2021_11:29:28
Name: 597, dtype: object
Ad_ID                                                 5680501
Ad_Link     /nepremicnine/lokacija-stanovanja-radomlje-ogl...
datetime                                  02.05.2021_11:29:29
Name: 600, dtype: object
Ad_ID                                                 6367551
Ad_Link     /nepremicnine/lokacija-stanovanja-center-oglas...
datetime                                  02.05.2021_11:29:29
Name: 602, dtype: object
Ad_ID                                                 3952649
Ad_Link     /nepremicnine/2-sobno-stanovanje-v-oplotnici-o...
datetime                        

Ad_ID                                                 6436318
Ad_Link     /nepremicnine/lokacija-stanovanja-studenci-102...
datetime                                  02.05.2021_11:29:37
Name: 847, dtype: object
Ad_ID                                                 6425777
Ad_Link     /nepremicnine/domzale-dragomelj-stanovanje-2.5...
datetime                                  02.05.2021_11:29:37
Name: 848, dtype: object
Ad_ID                                                 6425717
Ad_Link     /nepremicnine/pritlicno-stanovanje-izjemni-lok...
datetime                                  02.05.2021_11:29:37
Name: 849, dtype: object
Ad_ID                                                 5646709
Ad_Link     /nepremicnine/stanovanje-trisobno-benedikt-sv-...
datetime                                  02.05.2021_11:29:38
Name: 850, dtype: object
Ad_ID                                                 6422696
Ad_Link     /nepremicnine/lokacija-stanovanja-spodnja-sisk...
datetime                        

Ad_ID                                                 6010769
Ad_Link     /nepremicnine/stanovanje-celje-85.10-m2-novogr...
datetime                                  02.05.2021_11:29:45
Name: 1058, dtype: object
Ad_ID                                                 6010736
Ad_Link     /nepremicnine/stanovanje-vransko-127.98-m2-nov...
datetime                                  02.05.2021_11:29:45
Name: 1059, dtype: object
Ad_ID                                                 6010712
Ad_Link     /nepremicnine/stanovanje-vransko-108.00-m2-nov...
datetime                                  02.05.2021_11:29:45
Name: 1060, dtype: object
Ad_ID                                                 6010694
Ad_Link     /nepremicnine/stanovanje-vransko-112.77-m2-nov...
datetime                                  02.05.2021_11:29:45
Name: 1061, dtype: object
Ad_ID                                                 5967761
Ad_Link     /nepremicnine/prodaja-stanovanje-2-sobno-reslj...
datetime                    

Name: 1247, dtype: object
Ad_ID                                                 4311792
Ad_Link     /nepremicnine/stirisobno-stanovanje-piran-99.3...
datetime                                  02.05.2021_11:29:52
Name: 1248, dtype: object
Ad_ID                                                 4311791
Ad_Link     /nepremicnine/dvosobno-stanovanje-lucija-81-m2...
datetime                                  02.05.2021_11:29:52
Name: 1249, dtype: object
Ad_ID                                                 4311787
Ad_Link     /nepremicnine/del-hise-piran-169.9-m2-prodaja-...
datetime                                  02.05.2021_11:29:53
Name: 1250, dtype: object
Ad_ID                                                 4311778
Ad_Link     /nepremicnine/dvosobno-stanovanje-koper-62.5-m...
datetime                                  02.05.2021_11:29:53
Name: 1251, dtype: object
Ad_ID                                                 4267495
Ad_Link     /nepremicnine/izola-stanovanje-garsonjera-prod...
da

In [79]:
print(data_new)

Empty DataFrame
Columns: []
Index: []


In [73]:
data_all.iloc[0][0]

5784474

In [30]:
data_ads = scraping_ads(data_update)

  0%|                                                                                         | 0/1336 [00:08<?, ?it/s]


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59745): Max retries exceeded with url: /session/09d2daccd27beeb7041b24cfa7f37af9/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002107C20D310>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [58]:
date = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
data_ads.to_csv("./bolha_flats_all_info" + str(start) + '-'+ str(end) + '_' + str(date) + ".csv")

In [31]:
data['Ad_Link'][0]

'/nepremicnine/prodaja-stanovanje-trisobno-podravska-ruse-bezena-oglas-7006251'

In [94]:
data_ads['Cena']

0       \n\n192.000 €\n
1       \n\n134.500 €\n
2        \n\n29.500 €\n
3       \n\n120.000 €\n
4       \n\n180.000 €\n
             ...       
1183     \n\n60.000 €\n
1184    \n\n164.250 €\n
1185    \n\n150.015 €\n
1186    \n\n170.000 €\n
1187     \n\n90.000 €\n
Name: Cena, Length: 1188, dtype: object

In [4]:
data = pd.read_csv('bolha_flats_all_info1-49_14-03-2021_20-55-58.csv')
data.shape

(1188, 24)

In [6]:
data.head()

,Unnamed: 0,Šifra oglasa,Objavljen,Do poteka še,Število prikazov oglasa,Cena,Vrsta cene,Posredovanje,Lokacija,Umeščenost stanovanja,...,Neto površina,Leto izgradnje*,Leto zadnje prenove,Opremljenost in stanje,Energetski razred,Možnost menjave,Število parkirnih mest,Balkon / terasa / atrij,Zunanje površine in oprema,Interna številka oglasa
0,0,6523968,14.03.2021dne18:49,"\n29dni,22h,33min.\n",57-krat,\n\n192.000 €\n,Skupna cena,Prodam,"Savinjska, Celje, Slavko Šlander",V večstanovanjski stavbi,...,"105,00 m²",2004.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,6523894,14.03.2021dne18:39,"\n29dni,22h,24min.\n",29-krat,\n\n134.500 €\n,Skupna cena,Prodam,"Osrednjeslovenska, Ljubljana Center, Center",V večstanovanjski stavbi,...,"29,30 m²",1908.0,2020.0,Popolnoma opremljeno,EI ni potrebna (334. člen EZ-1),Možna menjava za drugo nepremičnino,NaN,NaN,NaN,NaN
2,2,6523589,14.03.2021dne18:07,"\n29dni,21h,52min.\n",33-krat,\n\n29.500 €\n,Skupna cena,Prodam,"Gorenjska, Tržič, Tržič",V večstanovanjski stavbi,...,"11,00 m²",1961.0,NaN,NaN,E (105 - 150 kWh/m2a),NaN,NaN,NaN,NaN,NaN
3,3,6523431,14.03.2021dne17:47,"\n29dni,21h,32min.\n",34-krat,\n\n120.000 €\n,Skupna cena,Prodam,"Savinjska, Slovenske Konjice, Slovenske Konjice",V večstanovanjski stavbi,...,"70,00 m²",2020.0,NaN,Neopremljeno,A2 (10 - 15 kWh/m2a),NaN,7+,"Balkon, Loža",NaN,NaN
4,4,6523190,14.03.2021dne17:16,"\n29dni,21h,0min.\n",23-krat,\n\n180.000 €\n,Skupna cena,Prodam,"Osrednjeslovenska, Ljubljana Bežigrad, Bežigrad",V večstanovanjski stavbi,...,NaN,1988.0,NaN,NaN,NaN,NaN,NaN,Zaprt balkon,NaN,NaN


In [10]:
data.columns


Index(['Unnamed: 0', 'Šifra oglasa', 'Objavljen ', 'Do poteka še',
       'Število prikazov oglasa', 'Cena', 'Vrsta cene', 'Posredovanje',
       'Lokacija', 'Umeščenost stanovanja', 'Nadstropje',
       'Skupno število nadstropij v stavbi', 'Vrsta stanovanja',
       'Bivalna površina', 'Neto površina', 'Leto izgradnje*',
       'Leto zadnje prenove', 'Opremljenost in stanje', 'Energetski razred',
       'Možnost menjave', 'Število parkirnih mest', 'Balkon / terasa / atrij',
       'Zunanje površine in oprema', 'Interna številka oglasa'],
      dtype='object')

In [20]:
data_feat = data.drop(columns=['Unnamed: 0', 'Šifra oglasa', 'Objavljen ', 'Do poteka še',
       'Število prikazov oglasa', 'Cena', 'Vrsta cene', 'Posredovanje','Interna številka oglasa','Možnost menjave', 'Zunanje površine in oprema'])
data_feat.head()

,Lokacija,Umeščenost stanovanja,Nadstropje,Skupno število nadstropij v stavbi,Vrsta stanovanja,Bivalna površina,Neto površina,Leto izgradnje*,Leto zadnje prenove,Opremljenost in stanje,Energetski razred,Število parkirnih mest,Balkon / terasa / atrij
0,"Savinjska, Celje, Slavko Šlander",V večstanovanjski stavbi,1.,2.0,3-sobno,"86,00 m²","105,00 m²",2004.0,NaN,NaN,NaN,NaN,NaN
1,"Osrednjeslovenska, Ljubljana Center, Center",V večstanovanjski stavbi,Pritličje,4.0,Garsonjera,"28,60 m²","29,30 m²",1908.0,2020.0,Popolnoma opremljeno,EI ni potrebna (334. člen EZ-1),NaN,NaN
2,"Gorenjska, Tržič, Tržič",V večstanovanjski stavbi,1.,5.0,Garsonjera,"11,00 m²","11,00 m²",1961.0,NaN,NaN,E (105 - 150 kWh/m2a),NaN,NaN
3,"Savinjska, Slovenske Konjice, Slovenske Konjice",V večstanovanjski stavbi,1.,5.0,2-sobno,NaN,"70,00 m²",2020.0,NaN,Neopremljeno,A2 (10 - 15 kWh/m2a),7+,"Balkon, Loža"
4,"Osrednjeslovenska, Ljubljana Bežigrad, Bežigrad",V večstanovanjski stavbi,7.,7.0,"1,5-sobno","53,50 m²",NaN,1988.0,NaN,NaN,NaN,NaN,Zaprt balkon


In [23]:
print(data_feat.shape)
print(data_feat.columns)

(1188, 13)
Index(['Lokacija', 'Umeščenost stanovanja', 'Nadstropje',
       'Skupno število nadstropij v stavbi', 'Vrsta stanovanja',
       'Bivalna površina', 'Neto površina', 'Leto izgradnje*',
       'Leto zadnje prenove', 'Opremljenost in stanje', 'Energetski razred',
       'Število parkirnih mest', 'Balkon / terasa / atrij'],
      dtype='object')
